In [32]:
import sys,os
sys.path.insert(0, f"{os.path.dirname(os.path.realpath('__file__'))}/../../../")
from rxnrecer.config import config as cfg
import pandas as pd
import numpy as np
from types import SimpleNamespace
from Bio import SeqIO
import torch
import argparse
import hashlib
import Reaction as rxnTool
import json
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
rxns = pd.read_feather(cfg.FILE_RHEA_REACTION)
rxns.head(2)

,reaction_id,equation,chebi_id,ec_number,equation_chebi,equation_smiles
0,RHEA:22636,dCTP + H2O = dCMP + diphosphate + H(+),CHEBI:61481;CHEBI:15377;CHEBI:57566;CHEBI:3301...,EC:3.6.1.9;EC:3.6.1.12;EC:3.6.1.65,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...,Nc1ccn([C@H]2C[C@H](O)[C@@H](COP([O-])(=O)OP([...
1,RHEA:22640,NADP(+) + sphinganine = 3-oxosphinganine + H(+...,CHEBI:58349;CHEBI:57817;CHEBI:58299;CHEBI:1537...,EC:1.1.1.102,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...


In [34]:
def build_rxn_files(rxn_row):
    """构建反应JSON文件，添加错误处理"""
    rxn_id = rxn_row['reaction_id']
    rxn_smiles = rxn_row['equation_smiles']
    rxn_equation = rxn_row['equation']
    rxn_equation_ref_chebi = rxn_row['equation_chebi']
    rxn_ec = rxn_row['ec_number']
    
    try:
        reaction = rxnTool.Reaction(rxn_smiles, rxn_equation, rxn_equation_ref_chebi, rxn_id=rxn_id, rxn_ec=rxn_ec)
        json_path = f'{cfg.DIR_RXN_JSON}{rxn_id.replace(":", "_")}.json'
        reaction.save_json_file(json_path)
        return True
    except Exception as e:
        print(f'处理失败 {rxn_id}: {e}')
        return False

# 测试单个反应
print("测试单个反应...")
result = build_rxn_files(rxn_row=rxns.iloc[0])
print(f"测试结果: {'成功' if result else '失败'}")

测试单个反应...
测试结果: 成功


In [35]:
# 处理所有反应并统计结果
print("开始处理所有反应...")
results = rxns.progress_apply(build_rxn_files, axis=1)

# 统计结果
success_count = results.sum()
total_count = len(results)
failed_count = total_count - success_count

print(f"\n处理完成!")
print(f"总数: {total_count}")
print(f"成功: {success_count} ({success_count/total_count*100:.2f}%)")
print(f"失败: {failed_count} ({failed_count/total_count*100:.2f}%)")

# 保存失败的反应ID
if failed_count > 0:
    failed_indices = results[~results].index
    failed_rxn_ids = rxns.loc[failed_indices, 'reaction_id'].tolist()
    
    with open('notebook_failed_reactions.txt', 'w') as f:
        for rxn_id in failed_rxn_ids:
            f.write(f'{rxn_id}\n')
    
    print(f"失败的反应ID已保存到: notebook_failed_reactions.txt")
    print(f"前10个失败的反应ID: {failed_rxn_ids[:10]}")

results

开始处理所有反应...


 45%|████▌     | 7389/16410 [01:57<01:46, 84.40it/s] 

警告: 无法创建产物分子 beta-hematin (SMILES: C1=2N3C(C=C4[N+]5=C(C=C6N7C8=CC9=[N+](C(=C1)C(=C9CCC([O-])=O)C)[Fe-2]573OC(CCC=\%10C=\%11C=C\%12C(=C(C\%13=CC\%14=[N+]\%15C(=CC=\%16N\%17C(=C(C\%16C=C)C)C=C([N+]\%11[Fe-2]\%15\%17(N\%13\%12)OC(CCC8=C6C)=O)C\%10C)C(=C\%14C=C)C)C)CCC([O-])=O)=O)C(=C4C)C=C)=C(C2C)C=C): 无法解析SMILES字符串: C1=2N3C(C=C4[N+]5=C(C=C6N7C8=CC9=[N+](C(=C1)C(=C9CCC([O-])=O)C)[Fe-2]573OC(CCC=\%10C=\%11C=C\%12C(=C(C\%13=CC\%14=[N+]\%15C(=CC=\%16N\%17C(=C(C\%16C=C)C)C=C([N+]\%11[Fe-2]\%15\%17(N\%13\%12)OC(CCC8=C6C)=O)C\%10C)C(=C\%14C=C)C)C)CCC([O-])=O)=O)C(=C4C)C=C)=C(C2C)C=C


 48%|████▊     | 7929/16410 [02:04<01:45, 80.05it/s]

解析反应失败 RHEA:55876: SMILES字符串为空
解析反应失败 RHEA:55920: SMILES字符串为空


...

 78%|███████▊  | 12846/16410 [03:08<00:36, 97.96it/s] 

解析反应失败 RHEA:75967: SMILES字符串为空


100%|██████████| 16410/16410 [03:50<00:00, 71.08it/s] 


处理完成!
总数: 16410
成功: 16410 (100.00%)
失败: 0 (0.00%)


0        True
1        True
2        True
3        True
4        True
         ... 
16405    True
16406    True
16407    True
16408    True
16409    True
Length: 16410, dtype: bool